In [9]:
from dotenv import load_dotenv
load_dotenv()

True

## See this for learning tools from scratch

https://python.langchain.com/docs/concepts/tools/

# Tavily Search tool

In [10]:
from langchain_tavily import TavilySearch

tool = TavilySearch(max_results=5)

In [13]:
# usage

tool.invoke("Who is the current Chairperson of HCLTECH?")

{'query': 'Who is the current Chairperson of HCLTECH?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.hcltech.com/leadership',
   'title': 'HCLTech Leadership Team – Visionaries Driving Innovation and ...',
   'content': 'Roshni Nadar Malhotra is the Chairperson of HCLTech, a leading global technology organization with a market capitalization of $55 billion, and the Chairperson',
   'score': 0.91288006,
   'raw_content': None},
  {'url': 'https://www.forbes.com/profile/roshni-nadar-malhotra/',
   'title': 'Roshni Nadar Malhotra - Forbes',
   'content': "Roshni Nadar Malhotra · Malhotra is the chairperson for HCLTech, a multinational IT services and consulting firm based in India. · She's responsible for all",
   'score': 0.86052185,
   'raw_content': None},
  {'url': 'https://en.wikipedia.org/wiki/HCLTech',
   'title': 'HCLTech - Wikipedia',
   'content': '(Chairperson) Shiv Nadar (Chairman Emeritus) C Vijayakumar (CEO & Managing Direct

In [15]:
# bind tool to the agent

from langchain.chat_models import init_chat_model
llm = init_chat_model("anthropic:claude-3-5-sonnet-latest")

llm_with_search_tool = llm.bind_tools([TavilySearch(max_results=5)])

In [16]:
# let LLm decide if it chooses to call or not call the tool

llm_with_search_tool.invoke("Hello, I wanted to find out how to make butter.")

AIMessage(content=[{'text': "I'll search for information about how to make butter at home.", 'type': 'text'}, {'id': 'toolu_015HTKY5nqyirpdxKk7fdCRG', 'input': {'query': 'how to make homemade butter step by step'}, 'name': 'tavily_search', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01AMeTmtYMX8DUAwg6LEdNnw', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1477, 'output_tokens': 77, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-5-sonnet-20241022'}, id='run--654c3bb7-895e-4b18-bab1-38df62c03914-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'how to make homemade butter step by step'}, 'id': 'toolu_015HTKY5nqyirpdxKk7fdCRG', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1477, 'output_tokens': 77, 'total_tokens': 1554, 'input_token_details': {'cache_read': 0, 'cache_creation': 

In [17]:
llm_with_search_tool.invoke("What do you know about Hitler?")

AIMessage(content=[{'text': 'Let me search for accurate historical information about Adolf Hitler.', 'type': 'text'}, {'id': 'toolu_011wVi6qHBJ9q37jShWogCoh', 'input': {'query': 'Adolf Hitler historical facts biography', 'search_depth': 'advanced'}, 'name': 'tavily_search', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_012hFjvFZZXYLMHrZNQSWfoZ', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1472, 'output_tokens': 89, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-5-sonnet-20241022'}, id='run--0241003b-8cb4-46cc-953f-b8ca7a062e5b-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'Adolf Hitler historical facts biography', 'search_depth': 'advanced'}, 'id': 'toolu_011wVi6qHBJ9q37jShWogCoh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1472, 'output_tokens': 89, 'total_tokens': 156

In [18]:
prompt = """
what is wrong with my python code snipped?
def myTool(x,y)
    return x + y
"""

llm_with_search_tool.invoke(prompt)

AIMessage(content="It seems you haven't provided the actual error you're encountering, but looking at the code snippet you shared, there is a syntax error. The function definition is missing a colon `:` after the parameters.\n\nHere's the correct syntax:\n\n```python\ndef myTool(x,y):\n    return x + y\n```\n\nThe colon `:` is required in Python syntax to indicate the beginning of a function's body. It's needed after function definitions, if statements, for loops, while loops, and other block statements in Python.\n\nThis is a basic syntax rule in Python - all compound statements (like function definitions) must end their header line with a colon, which tells Python that an indented block of code will follow.\n\nLet me know if you have any other questions about Python syntax or if you're encountering any other errors!", additional_kwargs={}, response_metadata={'id': 'msg_01XDnd4wNgk9CbJH9KFqn5G2', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 

## Tools based graph

In [19]:
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model
llm = init_chat_model("anthropic:claude-3-5-sonnet-latest")

from langchain_tavily import TavilySearch
llm_with_search_tool = llm.bind_tools([TavilySearch(max_results=5)])

# graph
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]
    

graph_builder = StateGraph(State)

def chatbot(state: State):
    return {"messages": [llm_with_search_tool.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

In [21]:
state = {
    "messages": [
        {
            "role": "user",
            "content": "Who is the current Prime Minister of NewZealand?"
        }
    ]
}

chatbot(state)

{'messages': [AIMessage(content=[{'text': 'Let me search for information about the current Prime Minister of New Zealand.', 'type': 'text'}, {'id': 'toolu_014mvt9nMgxUM4N6KVPRzBi7', 'input': {'query': 'Who is the current Prime Minister of New Zealand?'}, 'name': 'tavily_search', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01VwjvjxaGfyQAZq4qmAbryd', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1477, 'output_tokens': 78, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-5-sonnet-20241022'}, id='run--f37e3b34-50bc-4d55-8776-e5c3aeeb4186-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'Who is the current Prime Minister of New Zealand?'}, 'id': 'toolu_014mvt9nMgxUM4N6KVPRzBi7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1477, 'output_tokens': 78, 'total_tokens': 1555, 'input_tok